# Adaptation of {Doc2Vec Tutorial on the Lee Dataset}

In [1]:
import gensim
import os
import collections
import smart_open
import random
from __future__ import division
from __future__ import print_function

C:\ProgramData\Anaconda2\envs\py27\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## What is it?

Doc2Vec is an NLP tool for representing documents as a vector and is a generalizing of the Word2Vec method. This tutorial will serve as an introduction to Doc2Vec and present ways to train and assess a Doc2Vec model.
## Resources

* [Word2Vec Paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* [Doc2Vec Paper](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)
* [Dr. Michael D. Lee's Website](http://faculty.sites.uci.edu/mdlee)
* [Lee Corpus](http://faculty.sites.uci.edu/mdlee/similarity-data/)
* [IMDB Doc2Vec Tutorial](doc2vec-IMDB.ipynb)

## Getting Started

To get going, we'll need to have a set of documents to train our doc2vec model. In theory, a document could be anything from a short 140 character tweet, a single paragraph (i.e., journal article abstract), a news article, or a book. In NLP parlance a collection or set of documents is often referred to as a <b>corpus</b>. 

For this tutorial, we'll be training our model using the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included in gensim. This corpus contains 314 documents selected from the Australian Broadcasting
Corporation’s news mail service, which provides text e-mails of headline stories and covers a number of broad topics.

And we'll test our model by eye using the much shorter [Lee Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) which contains 50 documents.

In [22]:
# Set file names for train and test data
#test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = 'izip_new_data_all -train0.9.dat'
lee_test_file = 'izip_new_data_all -test0.1.dat'

## Define a Function to Read and Preprocess Text

Below, we define a function to open the train/test file (with latin encoding), read the file line-by-line, pre-process each line using a simple gensim pre-processing tool (i.e., tokenize text into individual words, remove punctuation, set to lowercase, etc), and return a list of words. Note that, for a given file (aka corpus), each continuous line constitutes a single document and the length of each line (i.e., document) can vary. Also, to train the model, we'll need to associate a tag/number with each document of the training corpus. In our case, the tag is simply the zero-based line number.

In [24]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="utf-8") as f:
        for i, line in enumerate(f):
            line_elems = line.split('\t')
            if tokens_only:
                yield gensim.utils.simple_preprocess(line_elems[-1])
            else:
                # For training data, add tags
                if len(line_elems) == 1:
                    print('ERROR:line without tag!')
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line_elems[-1]), line_elems[0:-1])

In [25]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file))

Let's take a look at the training corpus

In [26]:
#print lee_train_file
#print lee_test_file
print (train_corpus[29:31])
#for x in train_corpus[0].words:
 #   print x

[TaggedDocument(words=[u'solarcell', u'\ucc28\uc6d0', u'\ucc28\uc6d0', u'\uba40\ud2f0', u'\uba40\ud2f0\uad6c\uc870\uccb4', u'\uad6c\uc870\uccb4', u'\uacc4\uba74', u'\uc81c\uc5b4', u'\uae30\uc220', u'\uba40\ud2f0\uc2a4\ucf00\uc77c', u'\uc2a4\ucf00\uc77c', u'\ud0a4\ud14d', u'\uc120\ud0dd\uc801', u'\uce90\ub9ac\uc5b4', u'\uc804\ub2ec', u'\uc720\uae30', u'\ubb34\uae30', u'\ub098\ub178', u'\ub098\ub178\ubcf5\ud569\ubc15\ub9c9', u'\ubcf5\ud569', u'\ubc15\ub9c9', u'\uacc4\uba74\uc5d0\ub108\uc9c0', u'\uc5d0\ub108\uc9c0', u'\uce68\ud22c', u'\ubc29\uc9c0', u'\uc804\uc790', u'\uc804\uc790\uad6c\uc870', u'\uad6c\uc870', u'\uc5f0\uad6c', u'\ucd5c\uc885', u'\ucd5c\uc885\ubaa9\ud45c', u'\ubaa9\ud45c', u'\uc774\uc6a9', u'\uc544\ud0a4\ud14d\uccd0', u'\uc870\uc808', u'\uac1c\ubc1c', u'\uac00\ub2a5', u'\uad6c\ud604', u'\ub2e8\uacc4', u'\ub2e8\uacc4', u'\uae30\ubc18', u'\ub2ec\uc131', u'\ub2e4\uc74c', u'\ud575\uc2ec', u'\uc720\uae30\uc801', u'\uc5f0\uacc4', u'\ud615\uc0c1', u'\ub098\ub178\ub2e8\uc704\uccb

And the testing corpus looks like this:

In [7]:
#print(test_corpus[:1])

Notice that the testing corpus is just a list of lists and does not contain any tags.

## Training the Model

### Instantiate a Doc2Vec Object 

Now, we'll instantiate a Doc2Vec model with a vector size with 50 words and iterating over the training corpus 55 times. We set the minimum word count to 2 in order to give higher frequency words more weighting. Model accuracy can be improved by increasing the number of iterations but this generally increases the training time. Small datasets with short documents, like this one, can benefit from more training passes.

In [27]:
model = gensim.models.doc2vec.Doc2Vec(size=15, min_count=2, iter=20)
model.build_vocab(train_corpus)

### Build a Vocabulary

In [28]:
def tester(corpus):
    total_success = 0
    total_data = 0
    #WARNING : remove divide 4
    for doc_id in range(len(corpus)//4):
        inferred_vector = model.infer_vector(corpus[doc_id].words)
        sims = model.docvecs.most_similar([inferred_vector], topn=5)
        for tag in corpus[doc_id].tags:
            score = tag in [doc_tag for doc_tag, sim in sims]
            if score:
                total_success += 1
            total_data += 1

    print(total_success/total_data) 

In [29]:

def trial(the_size,the_iter):
    global model
    print("Trial with:",the_size,the_iter)
    model = gensim.models.doc2vec.Doc2Vec(size=the_size, min_count=2, iter=the_iter)
    model.build_vocab(train_corpus)
    model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)
    print('Train:',end='')
    tester(train_corpus)
    print('Test:',end='')
    tester(test_corpus)
for i in range(5,30,5):
    trial(the_size=15,the_iter=i)

Trial with: 15 5
Train:0.02858036925
Test:0.0234741784038
Trial with: 15 10
Train:0.0280588296652
Test:0.01220657277
Trial with: 15 15
Train:0.035673307604
Test:0.0225352112676
Trial with: 15 20
Train:0.053822885157
Test:0.0262910798122
Trial with: 15 25
Train:0.0755189318869
Test:0.0431924882629


Essentially, the vocabulary is a dictionary (accessible via `model.wv.vocab`) of all of the unique words extracted from the training corpus along with the count (e.g., `model.wv.vocab['penalty'].count` for counts for the word `penalty`).

### Time to Train

If the BLAS library is being used, this should take no more than 3 seconds.
If the BLAS library is not being used, this should take no more than 2 minutes, so use BLAS if you value your time.

In [104]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

Wall time: 24.2 s


35085663

### Inferring a Vector

One important thing to note is that you can now infer a vector for any piece of text without having to re-train the model by passing a list of words to the `model.infer_vector` function. This vector can then be compared with other vectors via cosine similarity.

In [72]:
model.infer_vector([u'데이터', u'알고리즘', u'소프트웨어'])
inferred_vector = model.infer_vector(train_corpus[0].words)
model.docvecs.most_similar([inferred_vector], topn=5)

[('EI05', 0.806117057800293),
 ('EE12', 0.8059660196304321),
 ('ED06', 0.8056091666221619),
 ('EA09', 0.7554200887680054),
 ('ED08', 0.735058069229126)]

## Assessing Model

To assess our new model, we'll first infer new vectors for each document of the training corpus, compare the inferred vectors with the training corpus, and then returning the rank of the document based on self-similarity. Basically, we're pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model. The expectation is that we've likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily. Additionally, we'll keep track of the second ranks for a comparison of less similar documents. 

In [11]:
tester(train_corpus)
tester(test_corpus)

0.351331592689
0.290622098422


Let's count how each document ranks with respect to the training corpus 

In [131]:
model.save('trained_with size=10&iter=195')

Basically, greater than 95% of the inferred documents are found to be most similar to itself and about 5% of the time it is mistakenly most similar to another document. the checking of an inferred-vector against a training-vector is a sort of 'sanity check' as to whether the model is behaving in a usefully consistent manner, though not a real 'accuracy' value.

This is great and not entirely surprising. We can take a look at an example:

In [18]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Notice above that the most similar document is has a similarity score of ~80% (or higher). However, the similarity score for the second ranked documents should be significantly lower (assuming the documents are in fact different) and the reasoning becomes obvious when we examine the text itself

In [19]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (94): «foreign minister alexander downer says the commonwealth democracy watchdog should put zimbabwe formally on its agenda in the first step to possible suspension from the organisation mr downer says ministers from the commonwealth ministerial action group cmag should review whether the reported violence and intimidation in zimbabwe means it has violated the commonwealth code of good governance cmag ministers from australia bangladesh barbados botswana britain canada malaysia and nigeria will meet in london tomorrow for talks on zimbabwe in recent meetings they have suspended both fiji and pakistan following military coups however their talks on the violent campaign of farm occupations in zimbabwe have been restricted to informal discussions as president robert mugabe government holds power through recognised elections mr downer also says the commonwealth ministers should maintain pressure on president mugabe to allow international observers to oversee presidential el

## Testing the Model

Using the same approach above, we'll infer the vector for a randomly chosen test document, and compare the document to our model by eye.

In [20]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus))
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (46): «the river elbe surged to an all time record high friday flooding more districts of the historic city of dresden as authorities scrambled to evacuate tens of thousands of residents in the worst flooding to hit central europe in memory in the czech republic authorities were counting the cost of the massive flooding as people returned to the homes and the vlava river receded revealing the full extent of the damage to lives and landmarks»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (9, 0.5827658772468567): «some roads are closed because of dangerous conditions caused by bushfire smoke motorists are being asked to avoid the hume highway between picton road and the illawarra highway where police have reduced the speed limit from kilometres an hour to in southern sydney picton road is closed between wilton and bulli appin road is closed from appin to bulli tops and all access roads to royal national park are closed motorists are also as

### Wrapping Up

That's it! Doc2Vec is a great way to explore relationships between documents.